# 🚀 Quick Financial Analysis Dashboard
Streamlined version for fast execution - Core analysis only

In [ ]:
# CELL 1: Quick Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("🚀 QUICK FINANCIAL ANALYSIS SETUP")
print("="*50)

# Try to load existing data first
if os.path.exists('transformed_financial_data.csv'):
    df = pd.read_csv('transformed_financial_data.csv', index_col=0, parse_dates=True)
    print(f"✅ Data loaded: {len(df)} rows")
else:
    # Create quick sample data
    dates = pd.date_range(start='2025-01-01', end='2025-08-26', freq='D')
    np.random.seed(42)
    
    price_base = 200
    returns = np.random.normal(0.001, 0.02, len(dates))
    prices = [price_base]
    
    for ret in returns[1:]:
        prices.append(prices[-1] * (1 + ret))
    
    df = pd.DataFrame({
        'Close': prices,
        'Open': [p * (1 + np.random.normal(0, 0.005)) for p in prices],
        'High': [p * (1 + abs(np.random.normal(0, 0.01))) for p in prices],
        'Low': [p * (1 - abs(np.random.normal(0, 0.01))) for p in prices],
        'Volume': np.random.randint(50000, 200000, len(dates))
    }, index=dates)
    
    # Add technical indicators
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['SMA_50'] = df['Close'].rolling(50).mean()
    df['RSI_14'] = 50 + np.random.normal(0, 15, len(df))
    df['MACD_Line'] = df['Close'].ewm(12).mean() - df['Close'].ewm(26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(9).mean()
    
    df = df.dropna()
    print(f"✅ Sample data created: {len(df)} rows")

print(f"📅 Date range: {df.index.min()} to {df.index.max()}")
print(f"📊 Columns: {list(df.columns)}")

In [ ]:
# CELL 2: Generate Trading Signals (Fast)
print("📊 GENERATING TRADING SIGNALS...")

# Simple signal generation
df['Signal'] = 0

# SMA Crossover: Buy when SMA_20 > SMA_50
sma_bullish = df['SMA_20'] > df['SMA_50']
df.loc[sma_bullish, 'Signal'] += 1

# RSI: Buy when oversold (< 30), Sell when overbought (> 70)
rsi_oversold = df['RSI_14'] < 30
rsi_overbought = df['RSI_14'] > 70
df.loc[rsi_oversold, 'Signal'] += 1
df.loc[rsi_overbought, 'Signal'] -= 1

# MACD: Buy when MACD > Signal
macd_bullish = df['MACD_Line'] > df['MACD_Signal']
df.loc[macd_bullish, 'Signal'] += 1

# Create final trading signal
df['Trade_Signal'] = 0
df.loc[df['Signal'] >= 2, 'Trade_Signal'] = 1  # Buy
df.loc[df['Signal'] <= -1, 'Trade_Signal'] = -1  # Sell

buy_signals = (df['Trade_Signal'] == 1).sum()
sell_signals = (df['Trade_Signal'] == -1).sum()

print(f"✅ Signals generated:")
print(f"   📈 Buy signals: {buy_signals}")
print(f"   📉 Sell signals: {sell_signals}")

In [ ]:
# CELL 3: Quick Backtesting
print("📈 RUNNING QUICK BACKTEST...")

# Calculate returns
returns = df['Close'].pct_change().fillna(0)
positions = df['Trade_Signal'].shift(1).fillna(0)
strategy_returns = positions * returns

# Calculate performance metrics
total_return = (strategy_returns + 1).cumprod().iloc[-1] - 1
buy_hold_return = (returns + 1).cumprod().iloc[-1] - 1
volatility = strategy_returns.std() * np.sqrt(252)
sharpe_ratio = (total_return * 252) / (volatility * 252) if volatility > 0 else 0

print(f"\n🏆 PERFORMANCE SUMMARY:")
print(f"   • Strategy Return: {total_return:.2%}")
print(f"   • Buy & Hold Return: {buy_hold_return:.2%}")
print(f"   • Volatility: {volatility:.2%}")
print(f"   • Sharpe Ratio: {sharpe_ratio:.3f}")
print(f"   • Total Trades: {abs(positions.diff()).sum() / 2:.0f}")

In [ ]:
# CELL 4: Essential Visualization (Fast)
print("📊 CREATING KEY VISUALIZATIONS...")

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Price chart with signals
ax1.plot(df.index, df['Close'], 'k-', alpha=0.8, label='Close Price')
ax1.plot(df.index, df['SMA_20'], 'blue', alpha=0.7, label='SMA 20')
ax1.plot(df.index, df['SMA_50'], 'red', alpha=0.7, label='SMA 50')

buy_points = df[df['Trade_Signal'] == 1]
sell_points = df[df['Trade_Signal'] == -1]
if len(buy_points) > 0:
    ax1.scatter(buy_points.index, buy_points['Close'], color='green', marker='^', s=60, label='Buy')
if len(sell_points) > 0:
    ax1.scatter(sell_points.index, sell_points['Close'], color='red', marker='v', s=60, label='Sell')

ax1.set_title('Price Chart with Trading Signals')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. RSI
ax2.plot(df.index, df['RSI_14'], 'purple')
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.7)
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.7)
ax2.set_title('RSI Indicator')
ax2.set_ylim(0, 100)
ax2.grid(True, alpha=0.3)

# 3. Performance comparison
strategy_cumulative = (strategy_returns + 1).cumprod()
buyhold_cumulative = (returns + 1).cumprod()
ax3.plot(df.index, (strategy_cumulative - 1) * 100, label='Strategy', color='blue')
ax3.plot(df.index, (buyhold_cumulative - 1) * 100, label='Buy & Hold', color='orange')
ax3.set_title('Strategy vs Buy & Hold (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Signal distribution
signal_counts = df['Signal'].value_counts().sort_index()
ax4.bar(signal_counts.index, signal_counts.values, alpha=0.7)
ax4.set_title('Signal Distribution')
ax4.set_xlabel('Signal Score')
ax4.set_ylabel('Frequency')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/quick_analysis_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualization complete!")

In [ ]:
# CELL 5: Summary Report
print("📋 FINAL SUMMARY REPORT")
print("="*50)

print(f"📊 Data Analysis:")
print(f"   • Dataset size: {len(df)} rows")
print(f"   • Date range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
print(f"   • Price range: ${df['Close'].min():.2f} - ${df['Close'].max():.2f}")

print(f"\n🎯 Trading Signals:")
print(f"   • Total buy signals: {buy_signals}")
print(f"   • Total sell signals: {sell_signals}")
print(f"   • Signal efficiency: {(buy_signals + sell_signals) / len(df) * 100:.1f}%")

print(f"\n💰 Performance:")
print(f"   • Strategy return: {total_return:.2%}")
print(f"   • Buy & hold return: {buy_hold_return:.2%}")
print(f"   • Alpha (outperformance): {(total_return - buy_hold_return):.2%}")
print(f"   • Risk-adjusted return (Sharpe): {sharpe_ratio:.3f}")

print(f"\n✅ Analysis completed in under 2 minutes!")
print(f"📊 Charts saved to plots/quick_analysis_dashboard.png")